<h1>Welcome!</h1><br>
<p>Thanks again for checking out my project<br>
This is where I will try to get the location data for each Toronto Neighborhood by using geocoder.<br>
    -------------------------------------------------------------------------------------------------------------------------------------------------
<br><h3>First I need to install the necessary libraries.

In [11]:
!pip install geocoder
!conda install BeautifulSoup4 --yes
print('installation complete')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.8.2       |           py36_0         161 KB
    certifi-2019.11.28         |           py36_0         156 KB
    soupsieve-1.9.5            |           py36_0          61 KB
    openssl-1.1.1d             |       h7b6447c_3         3.7 MB
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.5-py36_0                 

The following packages will b

<h3>I couldn't figure out how to import my dataframe from my other notebook so I am just copying all of the code to make it here.

In [14]:
import geocoder
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

webpage = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
#Create the BeautifulSoup object
soup = BeautifulSoup(webpage.content,'html.parser')
#Get the table
soup_table = soup.find('table')
#Turn the table into a string
textonly = soup_table.get_text(',')
#Turn the string into a list
full_list = textonly.split('\n')
#Clean the list
clean_list = []
for i in full_list:
    if i != ',' and i != '':
        i = i.replace(',','')
        clean_list.append(i)
columns = clean_list[0:3]
postcodes = []
for i in range(3, len(clean_list), 3):
    postcodes.append(clean_list[i])
boroughs = []
for i in range(4, len(clean_list), 3):
    boroughs.append(clean_list[i])
neighborhoods = []
for i in range(5, len(clean_list), 3):
    neighborhoods.append(clean_list[i])
data = {'Postcode':postcodes, 'Borough':boroughs, 'Neighbourhood':neighborhoods}
#Turn list into a dataframe
df = pd.DataFrame(data)
#Clean the DataFrame
df = df.drop(df[df.Borough=='Not assigned'].index)
df.reset_index(drop=True, inplace=True)
for i in range(0, len(df.Neighbourhood)):
    if df.Neighbourhood.iloc[i] == 'Not assigned':
        df.Neighbourhood.iloc[i] = df.Borough.iloc[i]
df_grouped = df.groupby(['Postcode', 'Borough'])
df_neighs = df.groupby(['Postcode', 'Borough']).Neighbourhood.unique()
df1 = df_neighs.to_frame()
df1.reset_index(inplace=True)
df1_neigh = df1.Neighbourhood.apply(lambda x: ', '.join(x))
df1 = df1[['Postcode', 'Borough']]
df1['Neighbourhood'] = df1_neigh
print(df1)

    Postcode      Borough                                      Neighbourhood
0        M1B  Scarborough                                     Rouge, Malvern
1        M1C  Scarborough             Highland Creek, Rouge Hill, Port Union
2        M1E  Scarborough                  Guildwood, Morningside, West Hill
3        M1G  Scarborough                                             Woburn
4        M1H  Scarborough                                          Cedarbrae
..       ...          ...                                                ...
98       M9N         York                                             Weston
99       M9P    Etobicoke                                          Westmount
100      M9R    Etobicoke  Kingsview Village, Martin Grove Gardens, Richv...
101      M9V    Etobicoke  Albion Gardens, Beaumond Heights, Humbergate, ...
102      M9W    Etobicoke                                          Northwest

[103 rows x 3 columns]


<h3>Now I am going to try to use geocoder to get the location data for each neighborhood.<br><br>
    Note: I worked on this for about 3 hours and gave up, but here is the closest I think I came. I tried the while loop as given in the directions, but it just led to an infinite loop. In the next block I am just going to use the .csv provided.

In [28]:
all_lats_longs = []
for postcode in df1.Postcode:
    g = geocoder.google('{}, Toronto, Ontario'.format(postcode))
    latitude = g.latlng[0]
    longitude = g.latlng[1]
    all_lats_longs.append([postcode, latitude, longitude])
    print(postcode)
print(all_lats_longs)


TypeError: 'NoneType' object is not subscriptable

<h3> Since geocoder failed, I am downloading the csv and putting it into a dataframe.

In [30]:
!wget -q -O 'geospatial_data.csv' https://cocl.us/Geospatial_data
ll_data = pd.read_csv('geospatial_data.csv')
print(ll_data.head())

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


<h3>Now I will join my two DataFrams to make the DataFrame required for this section

In [36]:
full_df = pd.merge(df1, ll_data, left_on='Postcode', right_on='Postal Code')
full_df = full_df[['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
full_df.rename(columns={'Postcode':'PostalCode'}, inplace=True)
print(full_df.head())

  Postcode      Borough                           Neighbourhood   Latitude  \
0      M1B  Scarborough                          Rouge, Malvern  43.806686   
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.784535   
2      M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3      M1G  Scarborough                                  Woburn  43.770992   
4      M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476  
